#  RAG 체인 구성

---

## RAG란 무엇인가?

### 🎯 핵심 개념
**Retrieval Augmented Generation (RAG)** 는 대규모 언어 모델(LLM)에 외부 지식을 연결하여 더 정확하고 최신의 정보를 제공하는 AI 프레임워크입니다.

### 🔍 RAG의 작동 원리
```
사용자 질문 → 관련 문서 검색 → 컨텍스트와 함께 LLM에 전달 → 답변 생성
```

### 📊 RAG vs 일반 LLM 비교
| 구분 | 일반 LLM | RAG |
|------|----------|-----|
| 정보 소스 | 사전 훈련 데이터만 | 외부 지식베이스 + 사전 훈련 데이터 |
| 최신성 | 훈련 시점까지 | 실시간 업데이트 가능 |
| 정확성 | 환각(hallucination) 가능성 | 검증된 문서 기반 답변 |
| 사용 사례 | 일반적인 질문 답변 | 특정 도메인의 전문적 답변 |

---

## 환경 설정

### 🛠️ 필수 라이브러리 설치

```bash
# 기본 LangChain 패키지
pip install langchain langchain-community langchain-core

# 텍스트 분할
pip install langchain-text-splitters

# 임베딩 모델
pip install langchain-openai langchain-huggingface

# 벡터 저장소
pip install langchain-chroma

# 문서 처리
pip install pypdf python-dotenv

# 웹 스크래핑
pip install beautifulsoup4

# 토크나이저
pip install tiktoken transformers sentence-transformers

# 실험적 기능 (SemanticChunker)
pip install langchain-experimental

### 🔑 환경 변수 설정

In [1]:
# .env 파일 생성
from dotenv import load_dotenv

load_dotenv()

# OpenAI API 키 설정 (필요시)
# OPENAI_API_KEY=your_openai_api_key_here

True

### 📋 기본 라이브러리 import


In [2]:
import os
from glob import glob
from pprint import pprint
import json
from pathlib import Path

---

## 문서 로더 (Document Loaders)

### 🎯 문서 로더란?
**Document Loader**는 다양한 소스에서 문서를 로드하여 LangChain의 `Document` 객체로 변환하는 도구입니다.

### 📄 Document 객체 구조
```python
from langchain_core.documents import Document

# Document 객체의 기본 구조
document = Document(
    page_content="문서의 텍스트 내용",
    metadata={
        "source": "문서 출처",
        "page": 1,
        "title": "문서 제목"
    }
)
```

### 📄 문서 로더의 종류
- PDF 파일 로더
- 웹 페이지 로더 
- CSV 데이터 로더
- 디렉토리 로더
- HTML 데이터 로더
- JSON 데이터 로더
- Markdown 데이터 로더
- Microsoft Office 데이터 로더


### 1. 🌐 웹 문서 로더 (WebBaseLoader)

In [7]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

# 기본 웹 문서 로드
web_loader = WebBaseLoader(
    web_paths=[
        "https://python.langchain.com/docs/tutorials/rag/",
        "https://js.langchain.com/docs/tutorials/rag/"
    ]
)

# 문서 로드
web_docs = web_loader.load()
print(f"로드된 문서 수: {len(web_docs)}")
print(f"첫 번째 문서 메타데이터: {web_docs[0].metadata}")

로드된 문서 수: 2
첫 번째 문서 메타데이터: {'source': 'https://python.langchain.com/docs/tutorials/rag/', 'title': 'Build a Retrieval Augmented Generation (RAG) App: Part 1 | 🦜️🔗 LangChain', 'description': 'One of the most powerful applications enabled by LLMs is sophisticated question-answering (Q&A) chatbots. These are applications that can answer questions about specific source information. These applications use a technique known as Retrieval Augmented Generation, or RAG.', 'language': 'en'}


In [10]:
print(web_docs[0].page_content)






Build a Retrieval Augmented Generation (RAG) App: Part 1 | 🦜️🔗 LangChain








Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as a retrieverHow to add memory to chatbotsHow to use example selectorsHow to add a semantic layer over graph databaseHow to invoke runnables in parallelHow to stream chat mod

In [11]:
# Beautiful Soup 파서 옵션 설정
web_loader_advanced = WebBaseLoader(
    web_paths=["https://python.langchain.com/docs/tutorials/rag/"],
    bs_kwargs={
        "parse_only": bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    }
)

# 문서 로드
web_docs_advanced = web_loader_advanced.load()
print(f"로드된 문서 수: {len(web_docs_advanced)}")
print(f"첫 번째 문서 메타데이터: {web_docs_advanced[0].metadata}")

로드된 문서 수: 1
첫 번째 문서 메타데이터: {'source': 'https://python.langchain.com/docs/tutorials/rag/'}


In [12]:
# 헤더 설정 (예: User-Agent)
web_loader_with_headers = WebBaseLoader(
    web_paths=["https://example.com"],
    header_template={
        "User-Agent": "Mozilla/5.0 (compatible; LangChain)"
    }
)

# 문서 로드
web_docs_with_headers = web_loader_with_headers.load()
print(f"로드된 문서 수: {len(web_docs_with_headers)}")
print(f"첫 번째 문서 메타데이터: {web_docs_with_headers[0].metadata}")

로드된 문서 수: 1
첫 번째 문서 메타데이터: {'source': 'https://example.com', 'title': 'Example Domain', 'language': 'No language found.'}


### 2. 📊 CSV 파일 로더 (CSVLoader)

In [14]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# 기본 CSV 로드
csv_loader = CSVLoader("./data/kbo_teams_2023.csv", encoding="utf-8")
csv_docs = csv_loader.load()

print(f"문서 수: {len(csv_docs)}")
print(f"첫 번째 문서:\n{csv_docs[0].page_content}")

문서 수: 10
첫 번째 문서:
Team: KIA 타이거즈
City: 광주
Founded: 1982
Home Stadium: 광주-기아 챔피언스 필드
Championships: 11
Introduction: KBO 리그의 전통 강호로, 역대 최다 우승 기록을 보유하고 있다. '타이거즈 스피릿'으로 유명하며, 양현종, 안치홍 등 스타 선수들을 배출했다. 광주를 연고로 하는 유일한 프로야구팀으로 지역 사랑이 강하다.


In [15]:
csv_docs[0].metadata

{'source': './data/kbo_teams_2023.csv', 'row': 0}

In [16]:
# 소스 컬럼 지정 및 인코딩 설정
csv_loader_advanced = CSVLoader(
    file_path="./data/kbo_teams_2023.csv",
    source_column="Team",      # 이 컬럼이 메타데이터의 source가 됨
    content_columns=["Introduction"],  # 이 컬럼이 문서의 내용이 됨
    metadata_columns=["Founded"],  # 이 컬럼이 메타데이터에 추가됨
    encoding="utf-8",          # 인코딩 명시
    csv_args={
        "delimiter": ",",      # 구분자
        "quotechar": '"',      # 인용 문자
    }
)

csv_docs_advanced = csv_loader_advanced.load()

# 문서 수와 첫 번째 문서 내용 출력
print(f"문서 수: {len(csv_docs_advanced)}")
print(f"첫 번째 문서 메타데이터: {csv_docs_advanced[0].metadata}")
print(f"첫 번째 문서 내용:\n{csv_docs_advanced[0].page_content}")

문서 수: 10
첫 번째 문서 메타데이터: {'source': 'KIA 타이거즈', 'row': 0, 'Founded': '1982'}
첫 번째 문서 내용:
Introduction: KBO 리그의 전통 강호로, 역대 최다 우승 기록을 보유하고 있다. '타이거즈 스피릿'으로 유명하며, 양현종, 안치홍 등 스타 선수들을 배출했다. 광주를 연고로 하는 유일한 프로야구팀으로 지역 사랑이 강하다.


### 3. 📖 PDF 파일 로더 

- **PyPDFLoader**

In [18]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화
pdf_loader = PyPDFLoader('./data/labor_law.pdf')

# 동기 로딩
pdf_docs = pdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

# 각 페이지별 정보 확인
for i, doc in enumerate(pdf_docs[:3]):
    print(f"페이지 {i+1}: {len(doc.page_content)} 문자")
    print(f"메타데이터: {doc.metadata}")

PDF 문서 개수: 20
페이지 1: 1811 문자
메타데이터: {'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'creationdate': '2024-10-15T14:45:34+09:00', 'moddate': '2024-10-15T14:45:34+09:00', 'source': './data/labor_law.pdf', 'total_pages': 20, 'page': 0, 'page_label': '1'}
페이지 2: 1709 문자
메타데이터: {'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'creationdate': '2024-10-15T14:45:34+09:00', 'moddate': '2024-10-15T14:45:34+09:00', 'source': './data/labor_law.pdf', 'total_pages': 20, 'page': 1, 'page_label': '2'}
페이지 3: 2164 문자
메타데이터: {'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'creationdate': '2024-10-15T14:45:34+09:00', 'moddate': '2024-10-15T14:45:34+09:00', 'source': './data/labor_law.pdf', 'total_pages': 20, 'page': 2, 'page_label': '3'}


In [19]:
print(pdf_docs[0].page_content[:1000])  # 첫 페이지의 내용 일부 출력

법제처                                                            1                                                       국가법령정보센터
근로기준법
 
근로기준법
[시행 2021. 11. 19.] [법률 제18176호, 2021. 5. 18., 일부개정]
고용노동부 (근로기준정책과 - 해고, 취업규칙, 기타) 044-202-7534
고용노동부 (근로기준정책과 - 소년) 044-202-7535
고용노동부 (근로기준정책과 - 임금) 044-202-7548
고용노동부 (여성고용정책과 - 여성) 044-202-7475
고용노동부 (임금근로시간정책과 - 근로시간, 휴게) 044-202-7545
고용노동부 (임금근로시간정책과 - 휴일, 연차휴가) 044-202-7973
고용노동부 (임금근로시간정책과 - 제63조 적용제외, 특례업종) 044-202-7530
고용노동부 (임금근로시간정책과 - 유연근로시간제) 044-202-7549
       제1장 총칙
 
제1조(목적) 이 법은 헌법에 따라 근로조건의 기준을 정함으로써 근로자의 기본적 생활을 보장, 향상시키며 균형 있는
국민경제의 발전을 꾀하는 것을 목적으로 한다.
 
제2조(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2018. 3. 20., 2019. 1. 15., 2020. 5. 26.>
1. “근로자”란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를 제공하는 사람을 말한다.
2. “사용자”란 사업주 또는 사업 경영 담당자, 그 밖에 근로자에 관한 사항에 대하여 사업주를 위하여 행위하는 자를
말한다.
3. “근로”란 정신노동과 육체노동을 말한다.
4. “근로계약”이란 근로자가 사용자에게 근로를 제공하고 사용자는 이에 대하여 임금을 지급하는 것을 목적으로 체
결된 계약을 말한다.
5. “임금”이란 사용자가 근로의 대가로 근로자에게 임금, 봉급, 그 밖에 어떠한 명칭으로든지 지급하는 모든 금품

- **다른 PDF 로더들**

In [20]:
from langchain_community.document_loaders import (
    UnstructuredPDFLoader,
    PyMuPDFLoader,
    PDFMinerLoader
)

# Unstructured PDF 로더 (이미지, 테이블 처리 가능) : 별도 의존성 설치 필요
# unstructured_loader = UnstructuredPDFLoader("./data/labor_law.pdf")

# PyMuPDF 로더 (빠른 처리)
pymupdf_loader = PyMuPDFLoader("./data/labor_law.pdf")

# PDFMiner 로더 (정확한 텍스트 추출)
pdfminer_loader = PDFMinerLoader("./data/labor_law.pdf")

In [21]:
# pymupdf_loader = PyMuPDFLoader("./data/labor_law.pdf")
pdf_docs = pymupdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

# 각 페이지별 정보 확인
for i, doc in enumerate(pdf_docs[:3]):
    print(f"페이지 {i+1}: {len(doc.page_content)} 문자")
    print(f"메타데이터: {doc.metadata}")

ImportError: pymupdf package not found, please install it with `pip install pymupdf`

In [ ]:
print(pdf_docs[0].page_content[:1000])  # 첫 페이지의 내용 일부 출력

In [ ]:
# pdfminer_loader = PDFMinerLoader("./data/labor_law.pdf")
pdf_docs = pdfminer_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

# 각 페이지별 정보 확인
for i, doc in enumerate(pdf_docs[:3]):
    print(f"페이지 {i+1}: {len(doc.page_content)} 문자")
    print(f"메타데이터: {doc.metadata}")

In [ ]:
print(pdf_docs[0].page_content[:1000])  # 첫 페이지의 내용 일부 출력

### 4. 📝 텍스트 파일 로더 (TextLoader)

In [22]:
from langchain_community.document_loaders import TextLoader

# 단일 텍스트 파일 로드
text_loader = TextLoader("./data/restaurant_menu.txt", encoding="utf-8")
text_docs = text_loader.load()

print(f"문서 수: {len(text_docs)}")
print(f"첫 번째 문서 내용:\n{text_docs[0].page_content[:1000]}")  # 첫 1000자 출력
print(f"첫 번째 문서 메타데이터: {text_docs[0].metadata}")

문서 수: 1
첫 번째 문서 내용:
1. 시그니처 스테이크
   • 가격: ₩35,000
   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스
   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.

2. 트러플 리조또
   • 가격: ₩22,000
   • 주요 식재료: 이탈리아산 아르보리오 쌀, 블랙 트러플, 파르미지아노 레지아노 치즈
   • 설명: 크리미한 텍스처의 리조또에 고급 블랙 트러플을 듬뿍 얹어 풍부한 향과 맛을 즐길 수 있는 메뉴입니다. 24개월 숙성된 파르미지아노 레지아노 치즈를 사용하여 깊은 맛을 더했으며, 주문 즉시 조리하여 최상의 상태로 제공됩니다.

3. 연어 타르타르
   • 가격: ₩18,000
   • 주요 식재료: 노르웨이산 생연어, 아보카도, 케이퍼, 적양파
   • 설명: 신선한 노르웨이산 생연어를 곱게 다져 아보카도, 케이퍼, 적양파와 함께 섞어 만든 타르타르입니다. 레몬 드레싱으로 상큼한 맛을 더했으며, 바삭한 브리오쉬 토스트와 함께 제공됩니다. 전채요리로 완벽한 메뉴입니다.

4. 버섯 크림 수프
   • 가격: ₩10,000
   • 주요 식재료: 양송이버섯, 표고버섯, 생크림, 트러플 오일
   • 설명: 양송이버섯과 표고버섯을 오랜 시간 정성스레 끓여 만든 크림 수프입니다. 부드러운 텍스처와 깊은 버섯 향이 특징이며, 최상급 트러플 오일을 살짝 뿌려 고급스러운 향을 더했습니다. 파슬리를 곱게 다져 고명으로 올려 제공됩니다.

5. 가든 샐러드
   • 가격: ₩12,000
   • 주요 식재료: 유기농 믹스 그린, 체리 토마토, 오이, 당근, 발사믹 드레싱
   • 설명: 신선한 유기농 채소들로 구성된 건강한 샐러드입니다. 아삭한 식감의 믹스 그린에 달콤한 체리 토마토, 오이, 당

In [23]:
# 디렉토리 내 모든 텍스트 파일 로드
from langchain_community.document_loaders import DirectoryLoader

directory_loader = DirectoryLoader(
    "./data/",
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)
all_text_docs = directory_loader.load()

print(f"전체 문서 수: {len(all_text_docs)}")
print(f"첫 번째 문서 내용:\n{all_text_docs[0].page_content[:1000]}")  # 첫 1000자 출력
print(f"첫 번째 문서 메타데이터: {all_text_docs[0].metadata}")

전체 문서 수: 2
첫 번째 문서 내용:
1. 시그니처 스테이크
   • 가격: ₩35,000
   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스
   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.

2. 트러플 리조또
   • 가격: ₩22,000
   • 주요 식재료: 이탈리아산 아르보리오 쌀, 블랙 트러플, 파르미지아노 레지아노 치즈
   • 설명: 크리미한 텍스처의 리조또에 고급 블랙 트러플을 듬뿍 얹어 풍부한 향과 맛을 즐길 수 있는 메뉴입니다. 24개월 숙성된 파르미지아노 레지아노 치즈를 사용하여 깊은 맛을 더했으며, 주문 즉시 조리하여 최상의 상태로 제공됩니다.

3. 연어 타르타르
   • 가격: ₩18,000
   • 주요 식재료: 노르웨이산 생연어, 아보카도, 케이퍼, 적양파
   • 설명: 신선한 노르웨이산 생연어를 곱게 다져 아보카도, 케이퍼, 적양파와 함께 섞어 만든 타르타르입니다. 레몬 드레싱으로 상큼한 맛을 더했으며, 바삭한 브리오쉬 토스트와 함께 제공됩니다. 전채요리로 완벽한 메뉴입니다.

4. 버섯 크림 수프
   • 가격: ₩10,000
   • 주요 식재료: 양송이버섯, 표고버섯, 생크림, 트러플 오일
   • 설명: 양송이버섯과 표고버섯을 오랜 시간 정성스레 끓여 만든 크림 수프입니다. 부드러운 텍스처와 깊은 버섯 향이 특징이며, 최상급 트러플 오일을 살짝 뿌려 고급스러운 향을 더했습니다. 파슬리를 곱게 다져 고명으로 올려 제공됩니다.

5. 가든 샐러드
   • 가격: ₩12,000
   • 주요 식재료: 유기농 믹스 그린, 체리 토마토, 오이, 당근, 발사믹 드레싱
   • 설명: 신선한 유기농 채소들로 구성된 건강한 샐러드입니다. 아삭한 식감의 믹스 그린에 달콤한 체리 토마토, 오이

### 🎯 실습 1: 웹 문서 로더 연습   ~ 14:53분까지

In [ ]:
# 다음 웹 페이지들을 로드하고 메타데이터를 출력해보세요
urls = [
    "https://python.langchain.com/docs/tutorials/",
    "https://python.langchain.com/docs/concepts/"
]

# 여기에 코드를 작성하세요

---

## 텍스트 분할 (Text Splitting)

### 🎯 텍스트 분할이 필요한 이유
1. **토큰 제한**: LLM은 입력 토큰 수에 제한이 있음
2. **검색 정확도**: 작은 청크가 더 정확한 검색 결과 제공
3. **메모리 효율성**: 대용량 문서의 효율적 처리

### 📊 분할 전략 비교
| 방법 | 장점 | 단점 | 사용 사례 |
|------|------|------|----------|
| CharacterTextSplitter | 단순, 빠름 | 문맥 고려 안함 | 간단한 텍스트 |
| RecursiveCharacterTextSplitter | 문맥 보존 우수 | 계산 복잡 | 일반적인 문서 |
| SemanticChunker | 의미 기반 분할 | 느림, 비용 많음 | 중요한 문서 |
| TokenTextSplitter | 정확한 토큰 수 | 토크나이저 의존 | API 비용 최적화 |

### 1. CharacterTextSplitter

In [24]:
long_text = pdf_docs[0].page_content
print(f'첫 번째 문서의 텍스트 길이: {len(long_text)}')

첫 번째 문서의 텍스트 길이: 1811


In [31]:
from langchain_text_splitters import CharacterTextSplitter

# 기본 설정
text_splitter = CharacterTextSplitter(
    separator="\n",        # 구분자
    chunk_size=1000,         # 청크 크기
    chunk_overlap=200,       # 중복 크기
    length_function=len,     # 길이 측정 함수
    is_separator_regex=False # 정규식 여부
)

# 텍스트 분할
chunks = text_splitter.split_text(long_text)
print(f"분할된 청크 수: {len(chunks)}")
for chunk in chunks:
    print(len(chunk))

분할된 청크 수: 3
1000
1000
210


In [28]:
import re
from langchain.text_splitter import CharacterTextSplitter

# 정규식을 사용한 문장 단위 분할
def create_improved_sentence_splitter():
    """
    법률 문서와 공문서에 적합한 문장 분할기를 생성합니다.
    """
    
    # 간단하고 안전한 정규식 패턴
    # 문장 끝 후 공백이 있고, 특정 패턴이 따라오지 않는 경우만 분할
    improved_pattern = r'(?<=[.!?])\s+(?!\s*(?:\d+|호|조|항|]|\)|[가-힣]{1,2}\s*\)|[A-Za-z]\s*\)|[,;:]))'
    
    sentence_splitter = CharacterTextSplitter(
        separator=improved_pattern,
        chunk_size=500,
        chunk_overlap=100,
        is_separator_regex=True,
        keep_separator=True
    )
    
    return sentence_splitter


# 정규식을 사용한 문장 단위 분할기 생성
sentence_splitter = create_improved_sentence_splitter()

# 문장 단위로 분할
sentence_chunks = sentence_splitter.split_text(long_text)
print(f"문장 단위로 분할된 청크 수: {len(sentence_chunks)}")
print(f"첫 번째 문장 청크 내용: {sentence_chunks[0]}")

Created a chunk of size 620, which is longer than the specified 500


문장 단위로 분할된 청크 수: 4
첫 번째 문장 청크 내용: 법제처                                                            1                                                       국가법령정보센터
근로기준법
 
근로기준법
[시행 2021. 11. 19.] [법률 제18176호, 2021. 5. 18., 일부개정]
고용노동부 (근로기준정책과 - 해고, 취업규칙, 기타) 044-202-7534
고용노동부 (근로기준정책과 - 소년) 044-202-7535
고용노동부 (근로기준정책과 - 임금) 044-202-7548
고용노동부 (여성고용정책과 - 여성) 044-202-7475
고용노동부 (임금근로시간정책과 - 근로시간, 휴게) 044-202-7545
고용노동부 (임금근로시간정책과 - 휴일, 연차휴가) 044-202-7973
고용노동부 (임금근로시간정책과 - 제63조 적용제외, 특례업종) 044-202-7530
고용노동부 (임금근로시간정책과 - 유연근로시간제) 044-202-7549
       제1장 총칙
 
제1조(목적) 이 법은 헌법에 따라 근로조건의 기준을 정함으로써 근로자의 기본적 생활을 보장, 향상시키며 균형 있는
국민경제의 발전을 꾀하는 것을 목적으로 한다.


In [29]:
print(f"두 번째 문장 청크 내용: {sentence_chunks[1]}")

두 번째 문장 청크 내용: 제2조(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2018. 3. 20., 2019. 1. 15., 2020. 5. 26.>
1. “근로자”란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를 제공하는 사람을 말한다.
2. “사용자”란 사업주 또는 사업 경영 담당자, 그 밖에 근로자에 관한 사항에 대하여 사업주를 위하여 행위하는 자를
말한다.
3. “근로”란 정신노동과 육체노동을 말한다.
4. “근로계약”이란 근로자가 사용자에게 근로를 제공하고 사용자는 이에 대하여 임금을 지급하는 것을 목적으로 체
결된 계약을 말한다.
5. “임금”이란 사용자가 근로의 대가로 근로자에게 임금, 봉급, 그 밖에 어떠한 명칭으로든지 지급하는 모든 금품을
말한다.
6. “평균임금”이란 이를 산정하여야 할 사유가 발생한 날 이전 3개월 동안에 그 근로자에게 지급된 임금의 총액을
그 기간의 총일수로 나눈 금액을 말한다.


### 2. RecursiveCharacterTextSplitter

- 재귀적으로 텍스트를 분할하여 문맥을 최대한 보존하는 분할기입니다.


In [33]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 기본 재귀 분할기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", r'(?<=[.!?])\s+(?!\s*(?:\d+|호|조|항|]|\)|[가-힣]{1,2}\s*\)|[A-Za-z]\s*\)|[,;:]))']  # 우선순위 순서
)

# Document 객체 분할
chunks = text_splitter.split_documents(pdf_docs)
print(f"생성된 청크 수: {len(chunks)}")

# 각 청크의 길이 확인
for i, chunk in enumerate(chunks[:3]):
    print(f"청크 {i+1}: {len(chunk.page_content)} 문자")

생성된 청크 수: 56
청크 1: 936 문자
청크 2: 944 문자
청크 3: 273 문자


### 3. 토큰 기반 분할

#### 🔧 TikToken 기반 분할

In [34]:
# OpenAI 토크나이저 사용
token_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",  # GPT-4 인코딩
    chunk_size=500,              # 토큰 수 기준
    chunk_overlap=100
)

chunks = token_splitter.split_documents([pdf_docs[0]])

# 토큰 수 확인
import tiktoken
tokenizer = tiktoken.get_encoding("cl100k_base")

for i, chunk in enumerate(chunks[:3]):
    token_count = len(tokenizer.encode(chunk.page_content))
    print(f"청크 {i+1}: {token_count} 토큰")

청크 1: 492 토큰
청크 2: 449 토큰
청크 3: 475 토큰


#### 🤗 Hugging Face 토크나이저

In [36]:
from transformers import AutoTokenizer

# BGE-M3 토크나이저 사용
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

hf_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=300,
    chunk_overlap=50
)

chunks = hf_splitter.split_documents([pdf_docs[0]])

# 토큰 수 확인
for i, chunk in enumerate(chunks[:3]):
    token_count = len(tokenizer(chunk.page_content)["input_ids"])
    print(f"청크 {i+1}: {token_count} 토큰")

c:\Users\User\ktds-llm\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


청크 1: 284 토큰
청크 2: 298 토큰
청크 3: 273 토큰


### 4. **Semantic Chunking**

- **SemanticChunker**는 텍스트를 의미 단위로 **분할**하는 특수한 텍스트 분할도구 

- 단순 길이 기반이 아닌 **의미 기반**으로 텍스트를 청크화하는데 효과적

- **breakpoint_threshold_type**: Text Splitting의 다양한 임계값(Threshold) 설정 방식 (통계적 기법) 

    - **Gradient** 방식: 임베딩 벡터 간의 **기울기 변화**를 기준으로 텍스트를 분할
    - **Percentile** 방식: 임베딩 거리의 **백분위수**를 기준으로 분할 지점을 결정 (기본값: 95%)
    - **Standard Deviation** 방식: 임베딩 거리의 **표준편차**를 활용하여 유의미한 변화점을 찾아서 분할
    - **Interquartile** 방식: 임베딩 거리의 **사분위수 범위**를 기준으로 이상치를 감지하여 분할

- 설치: pip install langchain_experimental 또는 uv add langchain_experimental


In [ ]:
from langchain_experimental.text_splitter import SemanticChunker 
from langchain_openai.embeddings import OpenAIEmbeddings

# 임베딩 모델을 사용하여 SemanticChunker를 초기화 
text_splitter = SemanticChunker(
    embeddings=OpenAIEmbeddings(model="text-embedding-3-small"),         # OpenAI 임베딩 사용
    breakpoint_threshold_type="gradient",  # 임계값 타입 설정 (gradient, percentile, standard_deviation, interquartile)
)

In [ ]:
chunks = text_splitter.split_documents([pdf_docs[0]])

print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")

### 5. 구조 기반 분할

#### 📄 HTML 분할

In [37]:
from langchain_text_splitters import HTMLHeaderTextSplitter

html_string = """
<!DOCTYPE html>
<html>
<body>
    <div>
        <h1>제목 1</h1>
        <p>내용 1</p>
        <h2>소제목 1.1</h2>
        <p>내용 1.1</p>
    </div>
</body>
</html>
"""

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
html_header_splits = html_splitter.split_text(html_string)

# 결과 출력
print(f"HTML 헤더로 분할된 청크 수: {len(html_header_splits)}")
for i, chunk in enumerate(html_header_splits):
    print(f"청크 {i+1}:\n{chunk}\n{'-'*40}")

HTML 헤더로 분할된 청크 수: 4
청크 1:
page_content='제목 1' metadata={'Header 1': '제목 1'}
----------------------------------------
청크 2:
page_content='내용 1' metadata={'Header 1': '제목 1'}
----------------------------------------
청크 3:
page_content='소제목 1.1' metadata={'Header 1': '제목 1', 'Header 2': '소제목 1.1'}
----------------------------------------
청크 4:
page_content='내용 1.1' metadata={'Header 1': '제목 1', 'Header 2': '소제목 1.1'}
----------------------------------------


#### 🧑‍💻 코드 분할

In [38]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter
)

# Python 코드 분할
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON,
    chunk_size=100,
    chunk_overlap=0
)

python_code = """
def hello_world():
    print("Hello, World!")

class MyClass:
    def __init__(self):
        self.value = 42
"""

python_docs = python_splitter.create_documents([python_code])

print(f"Python 코드 청크 수: {len(python_docs)}")
for i, doc in enumerate(python_docs):
    print(f"청크 {i+1}:\n{doc.page_content}\n{'-'*40}")

Python 코드 청크 수: 2
청크 1:
def hello_world():
    print("Hello, World!")
----------------------------------------
청크 2:
class MyClass:
    def __init__(self):
        self.value = 42
----------------------------------------


### 🎯 실습 2: 텍스트 분할 비교 ~ 15:47분까지

In [ ]:
# 다음 텍스트를 다양한 방법으로 분할하고 결과를 비교해보세요
sample_text = """
인공지능은 현대 기술의 핵심입니다. 
머신러닝을 통해 컴퓨터는 학습할 수 있습니다.

딥러닝은 신경망을 기반으로 합니다.
자연어 처리는 텍스트를 이해하는 기술입니다.

컴퓨터 비전은 이미지를 분석합니다.
강화학습은 행동을 통해 학습합니다.
"""

# 여기에 코드를 작성하세요

---

## 문서 임베딩 (Document Embedding)

### 🎯 임베딩이란?
텍스트를 고차원 벡터 공간의 숫자 배열로 변환하여 의미적 유사도를 계산할 수 있게 하는 기술입니다.

### 📊 임베딩 모델 비교
| 모델 | 차원 | 언어 지원 | 비용 | 성능 | 사용 사례 |
|------|------|----------|------|------|----------|
| OpenAI text-embedding-3-small | 1536 | 다국어 | 유료 | 높음 | 프로덕션 |
| OpenAI text-embedding-3-large | 3072 | 다국어 | 유료 | 최고 | 고성능 요구 |
| BAAI/bge-m3 | 1024 | 다국어 | 무료 | 높음 | 한국어 특화 |
| sentence-transformers/all-MiniLM-L6-v2 | 384 | 영어 | 무료 | 중간 | 로컬 개발 |

### 1. OpenAI 임베딩

#### 🔧 기본 설정

In [5]:
from langchain_openai import OpenAIEmbeddings

# 기본 임베딩 모델
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=1536,           # 차원 수 (기본값)
    show_progress_bar=True,    # 진행률 표시
    max_retries=3             # 재시도 횟수
)

print(f"임베딩 차원: {embeddings_model.dimensions}")
print(f"컨텍스트 길이: {embeddings_model.embedding_ctx_length}")

임베딩 차원: 1536
컨텍스트 길이: 8191


#### 📝 문서 임베딩

In [6]:
# 문서 컬렉션 임베딩
documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하는 기술입니다.",
    "컴퓨터 비전은 이미지를 분석하는 기술입니다."
]

# 배치 임베딩 (효율적)
doc_embeddings = embeddings_model.embed_documents(documents)
print(f"임베딩 벡터 수: {len(doc_embeddings)}")
print(f"각 벡터 차원: {len(doc_embeddings[0])}")

100%|██████████| 1/1 [00:01<00:00,  1.39s/it]

임베딩 벡터 수: 5
각 벡터 차원: 1536


In [7]:
# 쿼리 임베딩
query = "AI 기술에 대해 알려주세요"
query_embedding = embeddings_model.embed_query(query)
print(f"쿼리 임베딩 차원: {len(query_embedding)}")

100%|██████████| 1/1 [00:00<00:00,  1.47it/s]

쿼리 임베딩 차원: 1536


#### 💡 차원 축소 활용

In [42]:
# 비용 절약을 위한 차원 축소
compact_embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", 
    dimensions=512  # 원래 1536에서 512로 축소
)

# 성능과 비용의 균형점 찾는 것이 중요!!!
compact_doc_embeddings = compact_embeddings.embed_documents(documents)
print(f"축소된 임베딩 벡터 수: {len(compact_doc_embeddings)}")
print(f"축소된 각 벡터 차원: {len(compact_doc_embeddings[0])}") 

축소된 임베딩 벡터 수: 5
축소된 각 벡터 차원: 512


### 2. Hugging Face 임베딩

#### 🤗 BGE-M3 모델 (한국어 우수)


In [8]:
from langchain_huggingface import HuggingFaceEmbeddings

# BGE-M3 모델 (다국어, 한국어 성능 우수)
embeddings_bge = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={'device': 'cpu'},        # 'cuda' for GPU
    encode_kwargs={'normalize_embeddings': True}  # L2 정규화
)

# BGE-M3 모델로 문서 임베딩
bge_hf_embeddings = embeddings_bge.embed_documents(documents)
print(f"한국어 임베딩 차원: {len(bge_hf_embeddings[0])}")

한국어 임베딩 차원: 1024


#### 📱 경량 모델

In [9]:
# 빠른 처리를 위한 경량 모델
embedding_gte = HuggingFaceEmbeddings(
    model_name="Alibaba-NLP/gte-multilingual-base",
    model_kwargs={'device': 'cpu', 'trust_remote_code': True},  # trust_remote_code for custom models
    encode_kwargs={'normalize_embeddings': True}
)
    
# 경량 모델로 문서 임베딩
alibaba_hf_embeddings = embedding_gte.embed_documents(documents)
print(f"경량 모델 한국어 임베딩 차원: {len(alibaba_hf_embeddings[0])}") 

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- configuration.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of 

경량 모델 한국어 임베딩 차원: 768


### 3. Ollama 임베딩 (로컬)

In [10]:
from langchain_ollama import OllamaEmbeddings

# Ollama 서버가 실행 중이어야 함
embeddings_ollama = OllamaEmbeddings(
    model="bge-m3",                    # 사용할 모델
    # base_url="http://localhost:11434"  # Ollama 서버 주소
)

# 로컬 임베딩
local_embeddings = embeddings_ollama.embed_documents(documents)

print(f"로컬 임베딩 벡터 수: {len(local_embeddings)}")
print(f"각 벡터 차원: {len(local_embeddings[0])}")

로컬 임베딩 벡터 수: 5
각 벡터 차원: 1024


### 4. 유사도 계산 및 검색

#### 📏 코사인 유사도

In [12]:
from langchain_community.utils.math import cosine_similarity
import numpy as np

def find_most_similar(query, doc_embeddings, documents, embeddings_model):
    """가장 유사한 문서 찾기"""
    # 쿼리 임베딩
    query_embedding = embeddings_model.embed_query(query)
    
    # 코사인 유사도 계산
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]
    
    # 가장 유사한 문서 인덱스
    most_similar_idx = np.argmax(similarities)
    
    return {
        "document": documents[most_similar_idx],
        "similarity": similarities[most_similar_idx],
        "index": most_similar_idx
    }

# 쿼리와 문서 임베딩을 사용하여 가장 유사한 문서 찾기 (OpenAI)
query = "딥러닝에 대해 알려주세요"
result = find_most_similar(query, doc_embeddings, documents, embeddings_model)

print(f"쿼리: {query}")
print(f"가장 유사한 문서: {result['document']}")
print(f"유사도 점수: {result['similarity']:.4f}")

100%|██████████| 1/1 [00:01<00:00,  1.39s/it]

쿼리: 딥러닝에 대해 알려주세요
가장 유사한 문서: 딥러닝은 머신러닝의 한 종류입니다.
유사도 점수: 0.5868


In [13]:
# HuggingFaceEmbeddings를 사용한 유사도 검색 (BGE-M3)
result = find_most_similar(query, bge_hf_embeddings, documents, embeddings_bge)
print(f"쿼리: {query}")
print(f"가장 유사한 문서: {result['document']}")
print(f"유사도 점수: {result['similarity']:.4f}")

쿼리: 딥러닝에 대해 알려주세요
가장 유사한 문서: 딥러닝은 머신러닝의 한 종류입니다.
유사도 점수: 0.7360


In [14]:
# Alibaba-NLP/gte-multilingual-base 모델로 유사도 검색
result = find_most_similar(query, alibaba_hf_embeddings, documents, embedding_gte)
print(f"쿼리: {query}")
print(f"가장 유사한 문서: {result['document']}")
print(f"유사도 점수: {result['similarity']:.4f}")

쿼리: 딥러닝에 대해 알려주세요
가장 유사한 문서: 딥러닝은 머신러닝의 한 종류입니다.
유사도 점수: 0.7886


In [15]:
# Ollama 모델로 유사도 검색 (bge-m3)
result = find_most_similar(query, local_embeddings, documents, embeddings_ollama)
print(f"쿼리: {query}")
print(f"가장 유사한 문서: {result['document']}")
print(f"유사도 점수: {result['similarity']:.4f}")

쿼리: 딥러닝에 대해 알려주세요
가장 유사한 문서: 딥러닝은 머신러닝의 한 종류입니다.
유사도 점수: 0.7352


### 🎯 실습 3: 임베딩 모델 비교

In [ ]:
# 다음 질문들에 대해 다른 임베딩 모델들의 검색 성능을 비교해보세요
queries = [
    "기계학습이란 무엇인가요?",
    "이미지 인식 기술에 대해 설명해주세요",
    "언어 모델의 작동 원리는?"
]

# 여기에 코드를 작성하세요

---

## 벡터 저장소 (Vector Store)

### 🎯 벡터 저장소란?
임베딩된 벡터를 효율적으로 저장하고 유사도 기반 검색을 수행하는 특수 데이터베이스입니다.

### 📊 벡터 저장소 비교
| 종류 | 장점 | 단점 | 사용 사례 |
|------|------|------|----------|
| Chroma | 설치 간단, 로컬 친화적 | 대용량 처리 한계 | 개발, 프로토타입 |
| FAISS | 매우 빠름, 확장성 우수 | 설정 복잡 | 대용량 검색 |
| Pinecone | 완전 관리형, 고성능 | 유료, 클라우드 의존 | 프로덕션 |
| Weaviate | GraphQL 지원, 하이브리드 검색 | 학습 곡선 | 복합 검색 |

### 1. Chroma 벡터 저장소

#### 🚀 Chroma 설치 및 설정
```bash
pip install langchain-chroma
```

#### 📚 기본 사용법

In [4]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

# 문서 준비
pdf_loader = PyPDFLoader('./data/labor_law.pdf')
pdf_docs = pdf_loader.load()

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    chunk_size=500,
    chunk_overlap=100
)
chunks = text_splitter.split_documents(pdf_docs)

# 임베딩 모델
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma 벡터 저장소 생성
chroma_db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection_name="labor_law",
    persist_directory="./local_chroma_db",
    collection_metadata={"hnsw:space": "cosine"}  # 유사도 메트릭 (l2)
)

print(f"저장된 문서 수: {chroma_db._collection.count()}")

저장된 문서 수: 95


#### 💾 벡터 저장소 로드

In [5]:
# 기존 벡터 저장소 로드
chroma_db = Chroma(
    collection_name="labor_law",
    embedding_function=embeddings,
    persist_directory="./local_chroma_db"
)

print(f"로드된 문서 수: {chroma_db._collection.count()}")

로드된 문서 수: 95


#### 🔍 기본 검색 기능

In [9]:
doc.metadata

{'total_pages': 20,
 'creator': 'PyPDF',
 'creationdate': '2024-10-15T14:45:34+09:00',
 'moddate': '2024-10-15T14:45:34+09:00',
 'producer': 'iText 2.1.7 by 1T3XT',
 'page_label': '3',
 'page': 2,
 'source': './data/labor_law.pdf'}

In [7]:
# 1. 유사도 검색
query = "탄력 근로에 대해 설명해주세요"
similar_docs = chroma_db.similarity_search(
    query=query,
    k=5,  # 상위 5개 결과
    filter={"source": "./data/labor_law.pdf"}  # 메타데이터 필터
)

print(f"검색 결과 수: {len(similar_docs)}")
for i, doc in enumerate(similar_docs):
    print(f"결과 {i+1}: {doc.page_content[:500]}...")
    print("-" * 40)

검색 결과 수: 5
결과 1: [제목개정 2021. 5. 18.]
 
제49조(임금의 시효) 이 법에 따른 임금채권은 3년간 행사하지 아니하면 시효로 소멸한다.
 
       제4장 근로시간과 휴식
 
제50조(근로시간) ① 1주 간의 근로시간은 휴게시간을 제외하고 40시간을 초과할 수 없다.
② 1일의 근로시간은 휴게시간을 제외하고 8시간을 초과할 수 없다.
③ 제1항 및 제2항에 따라 근로시간을 산정하는 경우 작업을 위하여 근로자가 사용자의 지휘ㆍ감독 아래에 있는 대
기시간 등은 근로시간으로 본다.<신설 2012. 2. 1., 2020. 5. 26.>
 
제51조(3개월 이내의 탄력적 근로시간제) ① 사용자는 취업규칙(취업규칙에 준하는 것을 포함한다)에서 정하는 바에 따
라 2주 이내의 일정한 단위기간을 평균하여 1주 간의 근로시간이 제50조제1항의 근로시간을 초과하지 아니하는 범
위에서 특정한 주에 제50조제1항의 근로시간을, 특정한 날에 제50조제2항의 근로시간을 초과하여 근로하게 할 수...
----------------------------------------
결과 2: 법제처                                                            9                                                       국가법령정보센터
근로기준법
[제목개정 2021. 1. 5.]
 
제51조의2(3개월을 초과하는 탄력적 근로시간제) ① 사용자는 근로자대표와의 서면 합의에 따라 다음 각 호의 사항을
정하면 3개월을 초과하고 6개월 이내의 단위기간을 평균하여 1주간의 근로시간이 제50조제1항의 근로시간을 초과
하지 아니하는 범위에서 특정한 주에 제50조제1항의 근로시간을, 특정한 날에 제50조제2항의 근로시간을 초과하여
근로하게 할 수 있다. 다만, 특정한 주의 근로시간은 52시간을, 특정한 날의 근로시간은 12시간을 초과할 수 없다.
1. 대상 근로자의 범위
2. 단위기간(3개월을 초과하

In [10]:
# 2. 점수와 함께 검색 (유사도 점수 포함)
docs_with_scores = chroma_db.similarity_search_with_score(query, k=3)

for doc, score in docs_with_scores:
    print(f"점수: {score:.4f}")
    print(f"내용: {doc.page_content[:100]}...")
    print("-" * 50)

점수: 0.6994
내용: [제목개정 2021. 5. 18.]
 
제49조(임금의 시효) 이 법에 따른 임금채권은 3년간 행사하지 아니하면 시효로 소멸한다.
 
       제4장 근로시간과 휴식
 
제50...
--------------------------------------------------
점수: 0.7028
내용: 법제처                                                            9                                    ...
--------------------------------------------------
점수: 0.7457
내용: 동안의 소정근로시간에 비하여 짧은 근로자를 말한다.
② 제1항제6호에 따라 산출된 금액이 그 근로자의 통상임금보다 적으면 그 통상임금액을 평균임금으로 한다.
 
제3조(근로조건의 ...
--------------------------------------------------


### 2. FAISS 벡터 저장소

#### ⚡ FAISS 설치 및 사용
```bash
pip install faiss-cpu  # CPU 버전
# pip install faiss-gpu  # GPU 버전
```


In [11]:
len(chunks)

95

In [12]:
from langchain_community.vectorstores import FAISS

# FAISS 벡터 저장소 생성
faiss_db = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)

# 로컬 저장
faiss_db.save_local("./faiss_index")

# 로드
faiss_db = FAISS.load_local(
    "./faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)

# 검색
similar_docs = faiss_db.similarity_search(query, k=5)

print(f"검색 결과 수: {len(similar_docs)}")
for i, doc in enumerate(similar_docs):
    print(f"결과 {i+1}: {doc.page_content[:100]}...")
    print("-" * 40)

검색 결과 수: 5
결과 1: [제목개정 2021. 5. 18.]
 
제49조(임금의 시효) 이 법에 따른 임금채권은 3년간 행사하지 아니하면 시효로 소멸한다.
 
       제4장 근로시간과 휴식
 
제50...
----------------------------------------
결과 2: 법제처                                                            9                                    ...
----------------------------------------
결과 3: 동안의 소정근로시간에 비하여 짧은 근로자를 말한다.
② 제1항제6호에 따라 산출된 금액이 그 근로자의 통상임금보다 적으면 그 통상임금액을 평균임금으로 한다.
 
제3조(근로조건의 ...
----------------------------------------
결과 4: 통상적으로 소정근로시간을 초과하여 근로할 필요가 있는 경우에는 그 업무의 수행에 통상 필요한 시간을 근로한
것으로 본다.
② 제1항 단서에도 불구하고 그 업무에 관하여 근로자대표와...
----------------------------------------
결과 5: 한다.
 
제21조(전차금 상계의 금지) 사용자는 전차금(前借金)이나 그 밖에 근로할 것을 조건으로 하는 전대(前貸)채권과 임금
을 상계하지 못한다.
 
제22조(강제 저금의 금지...
----------------------------------------


### 3. 벡터 저장소 고급 기능

#### 🎛️ 메타데이터 필터링

In [14]:
# 복합 필터 조건
filter_criteria = {
    "$and": [
        {"source": {"$eq": "./data/labor_law.pdf"}},
        {"page": {"$gte": 10}}  # 10페이지 이상
    ]
}

filtered_results = chroma_db.similarity_search(
    query=query,
    k=5,
    filter=filter_criteria
)

print(f"필터링된 검색 결과 수: {len(filtered_results)}")
for i, doc in enumerate(filtered_results):
    print(f"결과 {i+1}: {doc.page_content[:100]}...")
    print(doc.metadata['page'])
    print("-" * 40)

필터링된 검색 결과 수: 5
결과 1: 통상적으로 소정근로시간을 초과하여 근로할 필요가 있는 경우에는 그 업무의 수행에 통상 필요한 시간을 근로한
것으로 본다.
② 제1항 단서에도 불구하고 그 업무에 관하여 근로자대표와...
10
----------------------------------------
결과 2: 쉬운 종류의 근로로 전환하여야 한다.<개정 2012. 2. 1.>
⑥ 사업주는 제1항에 따른 출산전후휴가 종료 후에는 휴가 전과 동일한 업무 또는 동등한 수준의 임금을 지급하는
직...
13
----------------------------------------
결과 3: 제59조(근로시간 및 휴게시간의 특례) ① 「통계법」 제22조제1항에 따라 통계청장이 고시하는 산업에 관한 표준의 중분
류 또는 소분류 중 다음 각 호의 어느 하나에 해당하는 사업...
10
----------------------------------------
결과 4: 법제처                                                            15                                   ...
14
----------------------------------------
결과 5: 서는 아니 된다.
⑦ 제2항에 따라 직장 내 괴롭힘 발생 사실을 조사한 사람, 조사 내용을 보고받은 사람 및 그 밖에 조사 과정에 참여
한 사람은 해당 조사 과정에서 알게 된 비밀...
14
----------------------------------------


#### 🔄 문서 업데이트

In [15]:
# 새 문서 추가
from langchain_core.documents import Document

new_docs = [Document(page_content="새로운 내용", metadata={"source": "new"})]
chroma_db.add_documents(new_docs)

# 문서 삭제 (ID 기반)
# chroma_db.delete(ids=["doc_id_1", "doc_id_2"])

# 전체 컬렉션 삭제
# chroma_db.delete_collection()

['ab922fa2-cb37-4c71-8d0e-4955947ec5a6']

### 🎯 실습 4: 벡터 저장소 구축 ~ 16:42분까지

In [ ]:
# 다음 단계로 나만의 벡터 저장소를 구축해보세요:
# 1. 웹에서 문서 로드
# 2. 적절한 크기로 분할
# 3. 임베딩 및 저장
# 4. 검색 테스트

# 여기에 코드를 작성하세요

---

## 검색기 (Retriever)

### 🎯 Retriever란?
벡터 저장소를 기반으로 사용자 질의에 가장 관련성 높은 문서를 검색하는 인터페이스입니다.

### 📊 검색 전략 비교
| 전략 | 설명 | 장점 | 단점 | 사용 사례 |
|------|------|------|------|----------|
| similarity | 단순 유사도 검색 | 빠름, 직관적 | 다양성 부족 | 일반적인 검색 |
| similarity_score_threshold | 임계값 기반 검색 | 품질 보장 | 결과 수 불안정 | 고품질 결과 필요 |
| mmr | 최대 한계 관련성 | 다양성 우수 | 느림 | 포괄적 정보 필요 |

### 1. 기본 유사도 검색

#### 🔍 Top-K 검색

In [16]:
# 벡터 저장소를 Retriever로 변환
retriever = chroma_db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  # 상위 5개 결과
)

# 검색 실행
query = "탄력 근로에 대해 설명해주세요"
retrieved_docs = retriever.invoke(query)

print(f"검색된 문서 수: {len(retrieved_docs)}")
for i, doc in enumerate(retrieved_docs):
    print(f"문서 {i+1}:")
    print(f"내용: {doc.page_content[:200]}...")
    print(f"출처: {doc.metadata.get('source', 'Unknown')}")
    print("-" * 50)

검색된 문서 수: 5
문서 1:
내용: [제목개정 2021. 5. 18.]
 
제49조(임금의 시효) 이 법에 따른 임금채권은 3년간 행사하지 아니하면 시효로 소멸한다.
 
       제4장 근로시간과 휴식
 
제50조(근로시간) ① 1주 간의 근로시간은 휴게시간을 제외하고 40시간을 초과할 수 없다.
② 1일의 근로시간은 휴게시간을 제외하고 8시간을 초과할 수 없다.
③ 제1항 및 제2항에 ...
출처: ./data/labor_law.pdf
--------------------------------------------------
문서 2:
내용: 법제처                                                            9                                                       국가법령정보센터
근로기준법
[제목개정 2021. 1. 5.]
 
제51조의2(3개월을 초과하는 탄력적 근로시간제) ① 사용자는 근로자대표와의 서면...
출처: ./data/labor_law.pdf
--------------------------------------------------
문서 3:
내용: 동안의 소정근로시간에 비하여 짧은 근로자를 말한다.
② 제1항제6호에 따라 산출된 금액이 그 근로자의 통상임금보다 적으면 그 통상임금액을 평균임금으로 한다.
 
제3조(근로조건의 기준) 이 법에서 정하는 근로조건은 최저기준이므로 근로 관계 당사자는 이 기준을 이유로 근로조건
을 낮출 수 없다.
 
제4조(근로조건의 결정) 근로조건은 근로자와 사용자가 동등한...
출처: ./data/labor_law.pdf
--------------------------------------------------
문서 4:
내용: 통상적으로 소정근로시간을 초과하여 근로할 필요가 있는 경우에는 그 업무의 수행에 통상 필요한 시간을 근로한
것으로 본다.
② 제1항 단서에도 불구하고 그 업무에 관하여 근로자대표와의 서면 합의를

### 2. 임계값 기반 검색

#### 📏 점수 임계값 설정

In [18]:
# 유사도 점수 임계값 기반 검색
threshold_retriever = chroma_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "score_threshold": 0.1,  # 0.3 이상의 유사도만
        "k": 10                  # 최대 10개까지
    }
)

retrieved_docs = threshold_retriever.invoke(query)

# 실제 유사도 점수 확인
from langchain_community.utils.math import cosine_similarity

for i, doc in enumerate(retrieved_docs):
    # 실제 유사도 계산
    doc_embedding = embeddings.embed_query(doc.page_content)
    query_embedding = embeddings.embed_query(query)
    similarity = cosine_similarity([query_embedding], [doc_embedding])[0][0]
    
    print(f"문서 {i+1} (유사도: {similarity:.4f}):")
    print(f"{doc.page_content[:100]}...")
    print()

문서 1 (유사도: 0.3006):
[제목개정 2021. 5. 18.]
 
제49조(임금의 시효) 이 법에 따른 임금채권은 3년간 행사하지 아니하면 시효로 소멸한다.
 
       제4장 근로시간과 휴식
 
제50...

문서 2 (유사도: 0.2973):
법제처                                                            9                                    ...

문서 3 (유사도: 0.2550):
동안의 소정근로시간에 비하여 짧은 근로자를 말한다.
② 제1항제6호에 따라 산출된 금액이 그 근로자의 통상임금보다 적으면 그 통상임금액을 평균임금으로 한다.
 
제3조(근로조건의 ...

문서 4 (유사도: 0.2529):
통상적으로 소정근로시간을 초과하여 근로할 필요가 있는 경우에는 그 업무의 수행에 통상 필요한 시간을 근로한
것으로 본다.
② 제1항 단서에도 불구하고 그 업무에 관하여 근로자대표와...

문서 5 (유사도: 0.2461):
한다.
 
제21조(전차금 상계의 금지) 사용자는 전차금(前借金)이나 그 밖에 근로할 것을 조건으로 하는 전대(前貸)채권과 임금
을 상계하지 못한다.
 
제22조(강제 저금의 금지...

문서 6 (유사도: 0.2332):
쉬운 종류의 근로로 전환하여야 한다.<개정 2012. 2. 1.>
⑥ 사업주는 제1항에 따른 출산전후휴가 종료 후에는 휴가 전과 동일한 업무 또는 동등한 수준의 임금을 지급하는
직...

문서 7 (유사도: 0.2273):
[본조신설 2021. 1. 5.]
 
제52조(선택적 근로시간제) ① 사용자는 취업규칙(취업규칙에 준하는 것을 포함한다)에 따라 업무의 시작 및 종료 시각
을 근로자의 결정에 맡기...

문서 8 (유사도: 0.2274):
법제처                                                            3                                    ...



### 3. MMR (Maximal Marginal Relevance) 검색

#### 🎯 다양성을 고려한 검색


In [19]:
# MMR 검색 - 관련성과 다양성의 균형
mmr_retriever = chroma_db.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 5,                # 최종 반환할 문서 수
        "fetch_k": 20,         # 초기 후보 문서 수
        "lambda_mult": 0.5     # 관련성 vs 다양성 (0=최대 다양성, 1=최대 관련성)
    }
)

mmr_docs = mmr_retriever.invoke(query)

print("MMR 검색 결과:")
for i, doc in enumerate(mmr_docs):
    print(f"문서 {i+1}: {doc.page_content[:150]}...")
    print()

MMR 검색 결과:
문서 1: [제목개정 2021. 5. 18.]
 
제49조(임금의 시효) 이 법에 따른 임금채권은 3년간 행사하지 아니하면 시효로 소멸한다.
 
       제4장 근로시간과 휴식
 
제50조(근로시간) ① 1주 간의 근로시간은 휴게시간을 제외하고 40시간을 초과할 수 없다.
②...

문서 2: 법제처                                                            1                                                       국가법령정보센터
근로기준법
 
근로기준법
[시행 2021...

문서 3: 제7조(강제 근로의 금지) 사용자는 폭행, 협박, 감금, 그 밖에 정신상 또는 신체상의 자유를 부당하게 구속하는 수단으로
써 근로자의 자유의사에 어긋나는 근로를 강요하지 못한다....

문서 4: 새로운 내용...

문서 5: 서는 아니 된다.
⑦ 제2항에 따라 직장 내 괴롭힘 발생 사실을 조사한 사람, 조사 내용을 보고받은 사람 및 그 밖에 조사 과정에 참여
한 사람은 해당 조사 과정에서 알게 된 비밀을 피해근로자등의 의사에 반하여 다른 사람에게 누설하여서는 아니 된
다. 다만, 조사와 관...



#### 🔧 lambda_mult 파라미터 실험


In [ ]:
# 다양한 lambda_mult 값으로 실험
lambda_values = [0.0, 0.25, 0.5, 0.75, 1.0]

for lambda_val in lambda_values:
    retriever = chroma_db.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": 3,
            "fetch_k": 10,
            "lambda_mult": lambda_val
        }
    )
    
    docs = retriever.invoke(query)
    print(f"\nLambda {lambda_val} 결과:")
    for i, doc in enumerate(docs):
        print(f"  {i+1}. {doc.page_content[:100]}...")
    
    print("-" * 40)

### 4. 고급 검색 기법

#### 🎛️ 메타데이터 필터링과 검색 결합

In [20]:
# 메타데이터 기반 필터링 retriever
filtered_retriever = chroma_db.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5,
        "filter": {
            "source": "./data/labor_law.pdf"
        }
    }
)

filtered_results = filtered_retriever.invoke(query)

print("메타데이터 기반 필터링 결과:")
for i, doc in enumerate(filtered_results):
    print(f"문서 {i+1}: {doc.page_content[:150]}...")
    print()

메타데이터 기반 필터링 결과:
문서 1: [제목개정 2021. 5. 18.]
 
제49조(임금의 시효) 이 법에 따른 임금채권은 3년간 행사하지 아니하면 시효로 소멸한다.
 
       제4장 근로시간과 휴식
 
제50조(근로시간) ① 1주 간의 근로시간은 휴게시간을 제외하고 40시간을 초과할 수 없다.
②...

문서 2: 법제처                                                            9                                                       국가법령정보센터
근로기준법
[제목개정 2021. 1. 5...

문서 3: 동안의 소정근로시간에 비하여 짧은 근로자를 말한다.
② 제1항제6호에 따라 산출된 금액이 그 근로자의 통상임금보다 적으면 그 통상임금액을 평균임금으로 한다.
 
제3조(근로조건의 기준) 이 법에서 정하는 근로조건은 최저기준이므로 근로 관계 당사자는 이 기준을 이유로 근...

문서 4: 통상적으로 소정근로시간을 초과하여 근로할 필요가 있는 경우에는 그 업무의 수행에 통상 필요한 시간을 근로한
것으로 본다.
② 제1항 단서에도 불구하고 그 업무에 관하여 근로자대표와의 서면 합의를 한 경우에는 그 합의에서 정하는 시간
을 그 업무의 수행에 통상 필요한 시...

문서 5: 한다.
 
제21조(전차금 상계의 금지) 사용자는 전차금(前借金)이나 그 밖에 근로할 것을 조건으로 하는 전대(前貸)채권과 임금
을 상계하지 못한다.
 
제22조(강제 저금의 금지) ① 사용자는 근로계약에 덧붙여 강제 저축 또는 저축금의 관리를 규정하는 계약을 체결하지...



#### 🔄 동적 검색 파라미터

In [ ]:
class DynamicRetriever:
    def __init__(self, vectorstore, embeddings):
        self.vectorstore = vectorstore
        self.embeddings = embeddings
    
    def retrieve(self, query, search_type="auto", k=5):
        """쿼리 특성에 따라 동적으로 검색 전략 선택"""
        
        # 쿼리 복잡도 분석
        query_length = len(query.split())
        
        if query_length <= 3:
            # 짧은 쿼리: 높은 임계값
            search_type = "similarity_score_threshold"
            search_kwargs = {"score_threshold": 0.25, "k": k}
        elif query_length > 10:
            # 긴 쿼리: MMR로 다양성 확보
            search_type = "mmr"
            search_kwargs = {"k": k, "fetch_k": k*3, "lambda_mult": 0.3}
        else:
            # 중간 길이: 기본 유사도 검색
            search_type = "similarity"
            search_kwargs = {"k": k}
        
        retriever = self.vectorstore.as_retriever(
            search_type=search_type,
            search_kwargs=search_kwargs
        )
        
        return retriever.invoke(query)

# 사용 예시
dynamic_retriever = DynamicRetriever(chroma_db, embeddings)

queries = [
    "탄력근로",  # 짧은 쿼리
    "탄력 근로에 대해 설명해주세요",  # 중간 쿼리
    "탄력 근로 제도의 장점과 단점, 그리고 실제 적용 사례를 포함하여 자세히 설명해주세요"  # 긴 쿼리
]

for query in queries:
    print(f"\n쿼리: {query}")
    print(f"길이: {len(query.split())} 단어")
    docs = dynamic_retriever.retrieve(query)
    print(f"검색 결과: {len(docs)}개 문서")
    print("-" * 40)

### 🎯 실습 5: 검색 전략 비교

In [ ]:
# 같은 질문에 대해 다른 검색 전략들의 결과를 비교해보세요
test_query = "근로시간 단축에 대한 규정은 무엇인가요?"

strategies = {
    "similarity": {"k": 5},
    "similarity_score_threshold": {"score_threshold": 0.3, "k": 10},
    "mmr": {"k": 5, "fetch_k": 15, "lambda_mult": 0.5}
}

# 여기에 코드를 작성하세요

---

## RAG 체인 구현

### 🎯 RAG 체인이란?
검색(Retrieval)과 생성(Generation)을 연결하여 외부 지식을 기반으로 답변을 생성하는 파이프라인입니다.

### 🔄 RAG 워크플로우
```
사용자 질문 → 관련 문서 검색 → 컨텍스트 구성 → LLM 프롬프트 → 답변 생성
```

### 1. 프롬프트 템플릿 설계

#### 📝 기본 RAG 프롬프트

In [21]:
from langchain_core.prompts import ChatPromptTemplate

# 기본 RAG 프롬프트 템플릿
basic_template = """주어진 컨텍스트를 기반으로 질문에 답변하세요.

컨텍스트:
{context}

질문: {question}

답변:"""

basic_prompt = ChatPromptTemplate.from_template(basic_template)

#### 🎨 고급 RAG 프롬프트

In [22]:

system_prompt = """당신은 전문적인 문서 분석 AI입니다. 주어진 **컨텍스트**를 바탕으로 사용자의 **질문**에 정확하고 유용한 답변을 제공하세요.

## 답변 지침
- 컨텍스트에 있는 정보만을 사용하여 답변하세요
- 확실하지 않은 정보는 "명확하지 않습니다"라고 명시하세요
- 답변은 논리적이고 구조화된 형태로 제공하세요
- 가능한 경우 구체적인 예시나 수치를 포함하세요
- 출처를 표시하세요

## 답변 형식
**핵심 답변:** (질문에 대한 직접적인 답변)

**세부 설명:** (추가적인 설명이나 배경 정보)

**관련 정보:** (컨텍스트에서 발견된 연관 정보)
"""

user_prompt = """
## 컨텍스트
{context}

## 질문
{question}
"""

advanced_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", user_prompt)
    ]
)

#### 🌟 도메인별 특화 프롬프트

In [ ]:
legal_template = """당신은 법률 문서 전문 AI입니다. 법률 조항을 정확히 해석하고 설명해주세요.

## 법률 해석 원칙
- 조문의 정확한 인용을 포함하세요
- 법적 용어는 일반인이 이해할 수 있도록 설명하세요
- 예외 조항이나 단서가 있다면 반드시 언급하세요
- 관련 법령이나 시행령도 함께 언급하세요

## 관련 법률 조항
{context}

## 법률 질의
{question}

## 법률 답변
**해당 조항:** (관련 법률 조항 인용)

**조항 해석:** (조항의 의미와 적용 범위)

**주의사항:** (예외 조항이나 제한 사항)

**답변:**"""

legal_prompt = ChatPromptTemplate.from_template(legal_template)

### 2. LLM 설정

#### 🤖 기본 모델 설정


In [26]:
from langchain_openai import ChatOpenAI

# 프로덕션용 설정
llm = ChatOpenAI(
    model="gpt-4.1",
    temperature=0.1,        # 일관성 있는 답변
    max_tokens=1000,        # 답변 길이 제한
    top_p=0.9,              # 다양성 제어
    frequency_penalty=0.1,  # 반복 방지
    presence_penalty=0.1    # 새로운 주제 장려
)

# 빠른 응답용 설정
fast_llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    max_tokens=500    # 답변 길이 제한
)

#### 🔧 모델별 최적 설정

In [ ]:
# 정확성 중시 (법률, 의료 등)
precise_llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,           # 가장 확실한 답변
    top_p=0.1,              # 보수적인 선택
    frequency_penalty=0,
    presence_penalty=0
)

# 창의성 중시 (마케팅, 콘텐츠 등)
creative_llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.7,        # 창의적 답변
    top_p=0.95,            # 다양한 선택
    frequency_penalty=0.2,
    presence_penalty=0.3
)

### 3. RAG 체인 구성

#### 🔗 기본 LCEL 체인

In [27]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    """문서 리스트를 문자열로 포맷"""
    return "\n\n".join([
        f"{doc.page_content}" 
        for i, doc in enumerate(docs)
    ])

# 기본 RAG 체인
basic_rag_chain = (
    RunnableParallel({
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    })
    | advanced_prompt
    | llm
    | StrOutputParser()
)

# 테스트
query = "탄력 근로에 대해 설명해주세요"
result = basic_rag_chain.invoke(query)
print(result)

**핵심 답변:**  
탄력적 근로시간제(탄력 근로)는 일정 기간(2주~6개월) 동안 근로시간을 평균하여 법정 근로시간(주 40시간, 일 8시간)을 맞추는 제도입니다. 특정 주나 특정일에 법정 근로시간을 초과해 일할 수 있지만, 전체 단위기간의 평균이 기준을 넘지 않아야 합니다.

**세부 설명:**  
탄력적 근로시간제는 업무량이 특정 시기에 집중되는 경우 등, 근로시간을 유연하게 운영할 필요가 있을 때 활용됩니다.  
- **3개월 이내 탄력적 근로시간제(제51조):**  
  - 취업규칙 등에 따라 2주 이내의 일정 단위기간을 평균하여 1주 근로시간이 40시간을 넘지 않으면, 특정 주·일에 법정 근로시간을 초과해 근무할 수 있습니다.
- **3개월 초과~6개월 이내 탄력적 근로시간제(제51조의2):**  
  - 근로자대표와 서면 합의가 필요합니다.
  - 단위기간(3~6개월) 동안 주 평균 40시간을 넘지 않아야 하며, 특정 주는 52시간, 특정 일은 12시간을 초과할 수 없습니다.
  - 근로일 종료 후 다음 근로일 시작 전까지 연속 11시간 이상의 휴식시간을 보장해야 합니다(불가피한 경우 제외).

**관련 정보:**  
- 제50조: 1주 40시간, 1일 8시간 초과 불가(휴게시간 제외)
- 제51조: 3개월 이내 탄력적 근로시간제
- 제51조의2: 3~6개월 탄력적 근로시간제, 주 52시간/일 12시간 한도, 연속 11시간 휴식
- 단위기간 내 평균이 기준을 넘지 않으면 특정 시기에 집중근무 가능

**출처:**  
- 근로기준법 제50조, 제51조, 제51조의2 (컨텍스트)


#### 🎯 고급 RAG 체인

In [28]:
from typing import Dict, List
import json

class AdvancedRAGChain:
    def __init__(self, retriever, llm, prompt_template):
        self.retriever = retriever
        self.llm = llm
        self.prompt_template = prompt_template
        
    def format_docs_with_metadata(self, docs: List) -> str:
        """메타데이터를 포함한 문서 포맷팅"""
        formatted_docs = []
        for i, doc in enumerate(docs):
            source = doc.metadata.get('source', 'Unknown')
            page = doc.metadata.get('page', 'N/A')
            
            formatted_doc = f"""
[문서 {i+1}]
출처: {source}
페이지: {page}
내용: {doc.page_content}
"""
            formatted_docs.append(formatted_doc)
        
        return "\n".join(formatted_docs)
    
    def retrieve_with_scores(self, query: str, k: int = 5) -> tuple:
        """점수와 함께 문서 검색"""
        docs_with_scores = self.retriever.vectorstore.similarity_search_with_score(query, k=k)
        docs = [doc for doc, score in docs_with_scores]
        scores = [score for doc, score in docs_with_scores]
        return docs, scores
    
    def invoke(self, query: str) -> Dict:
        """향상된 RAG 실행"""
        # 문서 검색
        docs, scores = self.retrieve_with_scores(query)

        # 일정 점수가 하한선 이하인 문서는 제외
        if not docs:
            return {
                "answer": "해당 쿼리에 대한 관련 문서를 찾을 수 없습니다.",
                "source_documents": [],
                "relevance_scores": [],
                "context": ""
            }   
        else:
            # 점수가 낮은 문서 제외
            docs = [doc for doc, score in zip(docs, scores) if score > 0.2]
            scores = [score for score in scores if score > 0.2]
        
        # 컨텍스트 구성
        context = self.format_docs_with_metadata(docs)
        
        # 프롬프트 구성 및 LLM 호출
        prompt = self.prompt_template.format(context=context, question=query)
        response = self.llm.invoke(prompt)
        
        return {
            "answer": response.content,
            "source_documents": docs,
            "relevance_scores": scores,
            "context": context
        }

# 고급 RAG 체인 사용
advanced_rag = AdvancedRAGChain(retriever, llm, advanced_prompt)
result = advanced_rag.invoke("탄력 근로에 대해 설명해주세요")

print("답변:", result["answer"])
print(f"\n검색된 문서 수: {len(result['source_documents'])}")
print(f"평균 관련도: {sum(result['relevance_scores'])/len(result['relevance_scores']):.4f}")

답변: **핵심 답변:**  
탄력적 근로시간제(탄력 근로)는 일정 기간(2주~6개월) 동안 근로시간을 평균하여 법정 근로시간(주 40시간, 일 8시간)을 초과하지 않는 범위 내에서, 특정한 날이나 주에 근로시간을 탄력적으로 조정할 수 있는 제도입니다. 3개월 이내와 3개월 초과 6개월 이내의 두 가지 유형이 있습니다.

**세부 설명:**  
1. **3개월 이내의 탄력적 근로시간제**  
   - 취업규칙 등에서 정한 바에 따라 2주 이내의 일정 단위기간을 평균하여 1주 근로시간이 40시간(제50조제1항)을 넘지 않는 범위에서, 특정 주 또는 특정일에 법정 근로시간을 초과하여 근로할 수 있습니다.  
   - (출처: 문서 1, p.7)

2. **3개월을 초과하는 탄력적 근로시간제(최대 6개월)**  
   - 근로자대표와의 서면 합의가 필요합니다.
   - 3개월 초과 6개월 이내의 단위기간을 평균하여 1주 근로시간이 40시간을 넘지 않는 범위에서, 특정 주에는 최대 52시간, 특정일에는 최대 12시간까지 근로할 수 있습니다.
   - 사용자는 근로일 종료 후 다음 근로일 개시 전까지 연속하여 11시간 이상의 휴식 시간을 부여해야 합니다(불가피한 경우 제외).
   - (출처: 문서 2, p.8)

**관련 정보:**  
- 탄력적 근로시간제는 취업규칙 또는 근로자대표와의 서면 합의 등 일정한 절차를 거쳐야 하며, 단위기간·근로시간 등 구체적인 사항을 명확히 정해야 합니다.
- 법정근로시간(주 40시간, 일 8시간)은 평균 기준이며, 특정 시기에 집중근무가 필요한 업종 등에 활용됩니다.
- (출처: 문서 1, p.7; 문서 2, p.8)

검색된 문서 수: 5
평균 관련도: 0.7298


### 🎯 실습 6: 완전한 RAG 시스템 구축      ~ 17:50분까지

In [ ]:
### 실습 1


# 다음 웹 페이지들을 로드하고 메타데이터를 출력해보세요
urls = [
    "https://python.langchain.com/docs/tutorials/",
    "https://python.langchain.com/docs/concepts/"
]

# 여기에 코드를 작성하세요
web_loader = WebBaseLoader(web_paths=urls)
web_docs = web_loader.load()
print(f"로드된 문서 수: {len(web_docs)}")
print(f"첫 번째 문서 메타데이터: {web_docs[0].metadata}")

In [ ]:
# 다음 요구사항을 만족하는 RAG 시스템을 구축해보세요:
# 1. 여러 문서 형식 지원 (PDF, 웹, 텍스트)
# 2. 동적 검색 전략 선택
# 3. 도메인별 프롬프트 템플릿
# 4. 응답 품질 평가

class ComprehensiveRAGSystem:
    def __init__(self):
        pass
    
    def load_documents(self, sources):
        """다양한 소스에서 문서 로드"""
        # 여기에 코드를 작성하세요
        pass
    
    def setup_vector_store(self, documents):
        """벡터 저장소 구성"""
        # 여기에 코드를 작성하세요
        pass
    
    def query(self, question, domain="general"):
        """도메인별 질의응답"""
        # 여기에 코드를 작성하세요
        pass

# 여기에 구현 코드를 작성하세요

---

## 실습 문제 예시 답안

In [ ]:
### 실습 1


# 다음 웹 페이지들을 로드하고 메타데이터를 출력해보세요
urls = [
    "https://python.langchain.com/docs/tutorials/",
    "https://python.langchain.com/docs/concepts/"
]

# 여기에 코드를 작성하세요
web_loader = WebBaseLoader(web_paths=urls)
web_docs = web_loader.load()
print(f"로드된 문서 수: {len(web_docs)}")
print(f"첫 번째 문서 메타데이터: {web_docs[0].metadata}")

In [ ]:
### 실습 2

# 다음 텍스트를 다양한 방법으로 분할하고 결과를 비교해보세요
sample_text = """
인공지능은 현대 기술의 핵심입니다. 
머신러닝을 통해 컴퓨터는 학습할 수 있습니다.

딥러닝은 신경망을 기반으로 합니다.
자연어 처리는 텍스트를 이해하는 기술입니다.

컴퓨터 비전은 이미지를 분석합니다.
강화학습은 행동을 통해 학습합니다.
"""

# 여기에 코드를 작성하세요
from langchain_text_splitters import CharacterTextSplitter
# 기본 문자 단위 분할기
char_splitter = CharacterTextSplitter(
    separator="\n\n",  # 두 줄 단위로 분할
    chunk_size=100,    # 청크 크기
    chunk_overlap=20,  # 중복 크기
    length_function=len,  # 길이 측정 함수
    is_separator_regex=False  # 정규식 여부
)
# 텍스트 분할
char_chunks = char_splitter.split_text(sample_text)
print(f"문자 단위 분할 결과: {len(char_chunks)} 청크")

In [ ]:
### 실습 3

# 다음 질문들에 대해 다른 임베딩 모델들의 검색 성능을 비교해보세요
queries = [
    "기계학습이란 무엇인가요?",
    "이미지 인식 기술에 대해 설명해주세요",
    "언어 모델의 작동 원리는?"
]

# 여기에 코드를 작성하세요
def compare_embedding_models(queries, doc_embeddings, documents, embeddings_models):
    """여러 임베딩 모델을 사용하여 쿼리와 문서 간 유사도 비교"""
    results = {}
    
    for model_name, model in embeddings_models.items():
        results[model_name] = []
        for query in queries:
            result = find_most_similar(query, doc_embeddings, documents, model)
            results[model_name].append({
                "query": query,
                "document": result["document"],
                "similarity": result["similarity"],
                "index": result["index"]
            })
    
    return results

# 임베딩 모델들 정의
embeddings_models = {
    "OpenAI": embeddings_model,
    "BGE-M3": embeddings_bge,
    "GTE": embedding_gte,
    "Ollama": embeddings_ollama
}   

# 모델 비교
comparison_results = compare_embedding_models(queries, doc_embeddings, documents, embeddings_models)


In [ ]:
### 실습 4

# 1. 웹에서 문서 로드
urls = [
    "https://python.langchain.com/docs/tutorials/",
    "https://python.langchain.com/docs/concepts/"
]
web_loader = WebBaseLoader(web_paths=urls)
web_docs = web_loader.load()
print(f"로드된 문서 수: {len(web_docs)}")

# 2. 적절한 크기로 분할
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    chunk_size=500,
    chunk_overlap=100
)
chunks = text_splitter.split_documents(web_docs)
print(f"분할된 청크 수: {len(chunks)}")

# 3. 임베딩 및 저장
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
chroma_db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection_name="web_docs",
    persist_directory="./web_chroma_db",
    collection_metadata={"hnsw:space": "cosine"}  # 유사도 메트릭
)
print(f"저장된 문서 수: {chroma_db._collection.count()}")

# 4. 검색 테스트
query = "LangChain 튜토리얼에 대해 알려주세요"
similar_docs = chroma_db.similarity_search(
    query=query,
    k=5,  # 상위 5개 결과
)
print(f"검색 결과 수: {len(similar_docs)}")
for i, doc in enumerate(similar_docs):
    print(f"결과 {i+1}: {doc.page_content[:100]}...")
    print("-" * 40)

In [ ]:
### 실습 5

# 같은 질문에 대해 다른 검색 전략들의 결과를 비교해보세요
test_query = "근로시간 단축에 대한 규정은 무엇인가요?"

strategies = {
    "similarity": {"k": 5},
    "similarity_score_threshold": {"score_threshold": 0.3, "k": 10},
    "mmr": {"k": 5, "fetch_k": 15, "lambda_mult": 0.5}
}

# 여기에 코드를 작성하세요
for strategy, params in strategies.items():
    print(f"\n검색 전략: {strategy}")
    retriever = chroma_db.as_retriever(
        search_type=strategy,
        search_kwargs=params
    )
    
    results = retriever.invoke(test_query)
    
    print(f"검색 결과 수: {len(results)}")
    for i, doc in enumerate(results):
        print(f"문서 {i+1}: {doc.page_content[:100]}...")
        print("-" * 40) 

In [ ]:
### 실습 6


# 다음 요구사항을 만족하는 RAG 시스템을 구축해보세요:
# 1. 여러 문서 형식 지원 (PDF, 웹, 텍스트)
# 2. 동적 검색 전략 선택
# 3. 도메인별 프롬프트 템플릿
# 4. 응답 품질 평가

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader, TextLoader

class ComprehensiveRAGSystem:
    def __init__(self):
        """RAG 시스템 초기화"""
        self.documents = []
        self.vector_store = None
        self.embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
        self.retriever = None
        self.llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.1)
    
    def load_documents(self, sources):
        """다양한 소스에서 문서 로드"""
        for source in sources:
            if source.endswith('.pdf'):
                loader = PyPDFLoader(source)
            elif source.startswith('http'):
                loader = WebBaseLoader(web_paths=[source])
            else:
                loader = TextLoader(source)
            
            docs = loader.load()
            self.documents.extend(docs)
    
    def setup_vector_store(self, documents):
        """벡터 저장소 구성"""
        self.vector_store = Chroma.from_documents(
            documents=documents,
            embedding=self.embeddings,
            collection_name="rag_system"
        )
    
    def query(self, question, domain="legal"):
        """도메인별 질의응답"""
        self.retriever = self.vector_store.as_retriever()
        context = self.retriever.invoke(question)

        # 도메인별 프롬프트 템플릿 선택
        if domain == "legal":
            prompt_template = ChatPromptTemplate.from_template(
                "주어진 컨텍스트를 기반으로 법률가 관점에서 질문에 답변하세요.\n\n"
                "컨텍스트:\n{context}\n\n"
                "질문: {question}\n\n"
                "답변:"
            )
        elif domain == "technical":
            prompt_template = ChatPromptTemplate.from_template(
                "주어진 컨텍스트를 기반으로 기술 전문가 관점에서 질문에 답변하세요.\n\n"
                "컨텍스트:\n{context}\n\n"
                "질문: {question}\n\n"
                "답변:"
            )
        else:
            prompt_template = ChatPromptTemplate.from_template(
                "주어진 컨텍스트를 기반으로 질문에 답변하세요.\n\n"
                "컨텍스트:\n{context}\n\n"
                "질문: {question}\n\n"
                "답변:"
            )

        # 프롬프트 구성 및 LLM 호출
        prompt = prompt_template.format(context=context, question=question)
        response = self.llm.invoke(prompt)
        return response

# 사용 예시
rag_system = ComprehensiveRAGSystem()
rag_system.load_documents([
    "./data/labor_law.pdf",
    "https://python.langchain.com/docs/tutorials/",
    "https://python.langchain.com/docs/concepts/"
])
rag_system.setup_vector_store(rag_system.documents)

result = rag_system.query("탄력 근로에 대해 설명해주세요", domain="legal")
print("답변:", result.content)

---